#### Building RAG Q/A Chatbot Using Langchain,GoogleGenAI and Chromadb  

In [2]:
#Importing required libraries 
from langchain_community.document_loaders import PyPDFLoader 
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain_google_genai import GoogleGenerativeAIEmbeddings 
from langchain_chroma import Chroma 


In [3]:
#Loading doucment 
loader = PyPDFLoader("yolov9_paper.pdf")
documents = loader.load()

In [4]:
len(documents)

18

In [5]:
print(documents[0].page_content)

YOLOv9: Learning What You Want to Learn
Using Programmable Gradient Information
Chien-Yao Wang1,2, I-Hau Yeh2, and Hong-Yuan Mark Liao1,2,3
1Institute of Information Science, Academia Sinica, Taiwan
2National Taipei University of Technology, Taiwan
3Department of Information and Computer Engineering, Chung Yuan Christian University, Taiwan
kinyiu@iis.sinica.edu.tw, ihyeh@emc.com.tw, and liao@iis.sinica.edu.tw
Abstract
Today’s deep learning methods focus on how to design
the most appropriate objective functions so that the pre-
diction results of the model can be closest to the ground
truth. Meanwhile, an appropriate architecture that can
facilitate acquisition of enough information for prediction
has to be designed. Existing methods ignore a fact that
when input data undergoes layer-by-layer feature extrac-
tion and spatial transformation, large amount of informa-
tion will be lost. This paper will delve into the important is-
sues of data loss when data is transmitted through deep net

In [6]:
#Splitting documents into chunks 
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap= 200)
docs = text_splitter.split_documents(documents= documents)

In [ ]:
#Arter splitting total number of documents
len(docs)

96

In [ ]:
#Create Embedding vector and store into chroma database 
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ["GOOGLE_API_KEY"] = "your api key"

In [13]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key= os.environ["GOOGLE_API_KEY"])

In [14]:
vector = embeddings.embed_query("Hello world")
vector[:5]

[0.05889486148953438,
 -0.004501754883676767,
 -0.06729809194803238,
 -0.012740520760416985,
 0.0645611360669136]

In [ ]:
len(vector)

768

In [16]:
vectorstores = Chroma.from_documents(
                                documents= docs,
                                embedding = embeddings
                            )

In [17]:
retriever = vectorstores.as_retriever(search_type = "similarity", search_kwargs= {"k": 3})

In [ ]:
os.environ["GROQ_API_KEY"] = "your api key"

In [19]:
#Create llm model 
from langchain_groq import ChatGroq 
llm = ChatGroq(model = "llama-3.3-70b-versatile",
               api_key = os.environ["GROQ_API_KEY"]
               )

In [21]:
#RAG Pipeline 


#create Prmpt template
from langchain_core.prompts import PromptTemplate

template =  """You are an expert assistant tasked with answering questions based on the provided documents.
Use only the given context to generate your answer.
If the answer cannot be found in the context, clearly state that you do not know.
Be detailed and precise in your response, but avoid mentioning or referencing the context itself.

Context:
{context}

Question:
{question}

Answer:"""

rag_prompt = PromptTemplate.from_template(template)

In [22]:
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.runnables import RunnablePassthrough 

rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        |rag_prompt
        |llm
        |StrOutputParser()
)

In [24]:
from IPython.display import Markdown 

response = rag_chain.invoke("What is reversible function?")
print(response)

A reversible function refers to a function that has an inverse transformation function. In other words, when a function `r` has an inverse transformation function `v`, it is called a reversible function. This means that the original data `X` can be recovered from the transformed data `rψ(X)` by applying the inverse function `vζ`, without losing any information. The equation `X = vζ(rψ(X))` represents this concept, where `ψ` and `ζ` are parameters of `r` and `v`, respectively.


In [25]:
Markdown(response)

A reversible function refers to a function that has an inverse transformation function. In other words, when a function `r` has an inverse transformation function `v`, it is called a reversible function. This means that the original data `X` can be recovered from the transformed data `rψ(X)` by applying the inverse function `vζ`, without losing any information. The equation `X = vζ(rψ(X))` represents this concept, where `ψ` and `ζ` are parameters of `r` and `v`, respectively.